In [ ]:
import os

import IPython.display as ipd
import nopdb
import numpy as np
import PIL
import torch

from spoof.dataset.dataset import FaceDataset
from spoof.model.vit import ViT

#### Helper functions

In [ ]:
# Define some functions to map the tensor back to an image
def inv_normalize(tensor):
    """Normalize an image tensor back to the 0-255 range."""
    tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min()) * (256 - 1e-5)
    return tensor


def inv_transform(tensor, normalize=True):
    """Convert a tensor to an image."""
    if normalize:
        tensor = inv_normalize(tensor)
    array = tensor.detach().cpu().numpy()
    array = array.transpose(1, 2, 0).astype(np.uint8)
    return PIL.Image.fromarray(array)

#### Load and visualize an image returned from the ```FaceDataset```

In [ ]:
# Setup dataset to get a random image and read it
os.chdir("..")
ds = FaceDataset("data/casia/test/annotations.csv")
img = ds[np.random.randint(len(ds))]["image"]

In [ ]:
# Show the image
inv_transform(img)

#### Load the model and predict

In [ ]:
model = ViT()

In [ ]:
def predict(input):
    """Run the model on an input and print the predicted classes with probabilities."""
    with torch.no_grad():
        score = model.get_liveness_score(model(input))
    return score

#### Capture and plot the attention weights

In [ ]:
def plot_attention(input, attn):
    """Given an input image and the attention tensor, plot the average attention weight given to each image patch by each attention head."""
    with torch.no_grad():
        # Loop over attention heads
        for h_weights in attn:
            h_weights = h_weights.mean(axis=-2)  # Average over all attention keys
            h_weights = h_weights[1:]  # Skip the [class] token
            plot_weights(input, h_weights)


def plot_weights(input, patch_weights):
    """Display the image, dimming each patch according to the given weight."""
    # Multiply each patch of the input image by the corresponding weight
    plot = inv_normalize(input.clone())
    for i in range(patch_weights.shape[0]):
        x = i * 16 % 224
        y = i // (224 // 16) * 16
        plot[:, y : y + 16, x : x + 16] *= patch_weights[i]
    ipd.display(inv_transform(plot, normalize=False))

###

In [ ]:
def get_attn(input, layer_idx):
    with nopdb.capture_call(
        model.extractor.encoder.layers[layer_idx].self_attention.forward
    ) as attn_call:
        predict(input).item()
    return attn_call

In [ ]:
# Get the attention tensor for the last layer
attn_call = get_attn(img, 11)

In [ ]:
attn_mat = attn_call.locals["attn_output_weights"][0]

In [ ]:
attn_mat